# <center> KITTIDATASET CLASS </center>

The following displays how the dataset is initialized and datapoints are visualized

In [ ]:
import os
import random
from matplotlib import pyplot as plt
os.chdir('..')

from data.kitti_dataset import KittiDataset

In [ ]:
kitti = KittiDataset() # initialize data object
print(kitti)

In [ ]:
# visualisation/ transformations check
idx = random.choice(range(kitti.num_files))

# while not any([_['name'] =='Truck' for _ in kitti.get_label(idx)]):
#   idx = random.choice(range(kitti.num_files))
  
print(idx)
plt.imshow(kitti.get_image(idx))
kitti.inspect_points(frame_idx=idx,downsample_voxel_size=0.05)

In [ ]:
# creates and load json file conatining cropped boxes
# kitti.save_cropped_boxes()
cropped_labels,cropped_cam_points = kitti.load_cropped_boxes()

In [ ]:
# visualizes isolated objects
kitti.vis_cropped_boxes(cropped_labels,cropped_cam_points,object_class='Pedestrian')

In [ ]:
kitti.vis_crop_aug_sampler()

# <center> GEO-OPT </center>

In [ ]:
from geoopt.manifolds import StereographicExact
from geoopt import ManifoldTensor, ManifoldParameter
import torch
import numpy as np


manifold    = StereographicExact(k=-0.01,learnable=True)
velo_points = kitti.get_velo_points(idx)
velo_points = torch.from_numpy(velo_points.xyz).double()
velo_points = ManifoldTensor(velo_points, manifold=manifold)

In [ ]:
print(velo_points)
velo_points.proj_()
print(velo_points)

In [ ]:
points_eucl = torch.linspace(0,100,100)[::,torch.newaxis]*torch.tensor([1,1,1])
diffs_eucl  = points_eucl[1:] - points_eucl[:-1]
dist_eucl   = torch.linalg.norm(diffs_eucl, axis=1)
print(dist_eucl)

points_stereo = ManifoldTensor(points_eucl,manifold=manifold)
points_stereo.proj_()
# diffs_stereo  = points_stereo[1:] - points_stereo[:-1]
# dist_stereo   = torch.linalg.norm(diffs_stereo, axis=1)
# print(dist_stereo)